In [1]:
from config import *
from data_preprocessing import *
from utils import *
from models import *
from train import *
from evaluate import *

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

set_seeds(RANDOM_SEED)


In [2]:
df = load_and_prepare_data(DATA_PATH)
train, val, test = split_data(df)

print(df.shape)
print(df.head())


(2880844, 20)
   weekday    server_timestamp  voltage  current  power  frequency  energy  \
0        3 2021-08-05 12:44:50    119.9     1.09  118.7       60.0  171.89   
1        3 2021-08-05 12:44:51    119.9     1.05  121.3       60.0  171.89   
2        3 2021-08-05 12:44:52    119.9     1.01  117.5       59.9  171.89   
3        3 2021-08-05 12:44:53    119.9     1.02  119.2       59.9  171.89   
4        3 2021-08-05 12:44:54    120.0     0.91  106.5       59.9  171.89   

   power_factor  sensor_temperature  cpu_usage_percent  WORKSTATION_CPU_POWER  \
0          0.90               33.33                0.0                    0.0   
1          0.97               33.89                0.0                    0.0   
2          0.97               33.33                0.0                    0.0   
3          0.97               33.33                0.0                    0.0   
4          0.97               33.33                0.0                    0.0   

   WORKSTATION_CPU_TEMP  WORKS

In [3]:
feature_cols = [
    c for c in train.columns
    if c not in ["server_timestamp", "energy", TARGET_COLUMN]
]

train, val, test, scaler = scale_numeric_features(
    train, val, test, feature_cols
)


In [4]:
X_train, y_train = create_sequences(
    train[feature_cols].values,
    train[TARGET_COLUMN].values,
    SEQUENCE_LENGTH
)

X_val, y_val = create_sequences(
    val[feature_cols].values,
    val[TARGET_COLUMN].values,
    SEQUENCE_LENGTH
)

X_test, y_test = create_sequences(
    test[feature_cols].values,
    test[TARGET_COLUMN].values,
    SEQUENCE_LENGTH
)

seq_len = X_train.shape[1]
num_feats = X_train.shape[2]


In [5]:
import os
os.makedirs("data/splits", exist_ok=True)

np.save("data/splits/X_train.npy", X_train)
np.save("data/splits/y_train.npy", y_train)

np.save("data/splits/X_val.npy", X_val)
np.save("data/splits/y_val.npy", y_val)

np.save("data/splits/X_test.npy", X_test)
np.save("data/splits/y_test.npy", y_test)


In [ ]:
hyperparameter_bounds = {
    'lstm_layers': ([2, 3], 'int'),
    'units': ([64, 128], 'categorical'),
    'dropout': ([0.3, 0.6], 'float'),
    'optimizer': (['adamw'], 'categorical'),
    'learning_rate': ([5e-5, 5e-4], 'float_log'),
    'batch_size': ([32], 'categorical')
}

ninja_optimizer = NinjaOptimizationAlgorithm(
    objective_function=lambda p: objective_function_lstm(
        p, X_train, y_train, X_val, y_val
    ),
    bounds=hyperparameter_bounds,
    n_agents=N_AGENTS,
    max_iterations=MAX_ITERATIONS,
    exploration_factor=EXPLORATION_FACTOR,
    exploitation_factor=EXPLOITATION_FACTOR
)

best_params, best_loss, convergence = ninja_optimizer.optimize()


Starting Ninja Optimization Algorithm...



In [ ]:
print("Best params from NinOA:", best_params)


In [ ]:
if best_params is None:
    raise ValueError(
        "best_params is None. NinOA did not find a valid configuration."
    )

FINAL_EPOCHS = 40
FINAL_PATIENCE = 7

final_model = create_lstm_model(
    best_params,
    seq_len,
    num_feats
)

from tensorflow.keras.callbacks import EarlyStopping

final_early_stop = EarlyStopping(
    monitor='val_loss',
    patience=FINAL_PATIENCE,
    restore_best_weights=True,
    verbose=1
)

history = final_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=FINAL_EPOCHS,
    batch_size=best_params['batch_size'],
    callbacks=[final_early_stop],
    verbose=1
)


In [ ]:
results_df, y_true, y_pred, explanation = evaluate_model(
    final_model,
    X_test,
    y_test
)

print(results_df)
print(explanation)


In [ ]:
# =======================
# Visualisation & Analysis Plots
# =======================

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

# -------- Plot 1: Predicted vs Actual --------
plt.figure(figsize=(7, 6))
plt.scatter(y_true, y_pred, alpha=0.5)
plt.plot(
    [y_true.min(), y_true.max()],
    [y_true.min(), y_true.max()],
    linestyle="--"
)
plt.xlabel("Actual Δenergy")
plt.ylabel("Predicted Δenergy")
plt.title("Predicted vs Actual Energy Increment (Δenergy)")
plt.tight_layout()
plt.show()


# -------- Plot 2: Residual Distribution --------
residuals = y_true - y_pred

plt.figure(figsize=(7, 5))
sns.histplot(residuals, bins=40, kde=True)
plt.xlabel("Residual (Actual − Predicted Δenergy)")
plt.title("Residual Distribution")
plt.tight_layout()
plt.show()


# -------- Plot 3: Training vs Validation Loss --------
plt.figure(figsize=(7, 5))
plt.plot(history.history['loss'], label="Training Loss")
plt.plot(history.history['val_loss'], label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("Training vs Validation Loss")
plt.legend()
plt.tight_layout()
plt.show()


# -------- Plot 4: Ninja Optimization Convergence --------
plt.figure(figsize=(7, 5))
plt.plot(convergence, marker='o')
plt.xlabel("Iteration")
plt.ylabel("Best Validation Loss")
plt.title("Ninja Optimization Algorithm Convergence")
plt.tight_layout()
plt.show()


In [ ]:
os.makedirs("results", exist_ok=True)

import json
with open("results/best_params.json", "w") as f:
    json.dump(best_params, f, indent=4)


results_df.to_csv("results/evaluation_metrics.csv", index=False)

final_model.save("NiOA_DRNN_final_model.h5")


In [ ]:
# =======================
# Save test predictions for statistical tests
# =======================

import numpy as np
import os

os.makedirs("results/predictions", exist_ok=True)

# Save ground truth once
np.save("results/predictions/y_test.npy", y_true)

# Save NinOA + DRNN predictions
np.save("results/predictions/y_pred_ninoa_drnn.npy", y_pred)
